# In this notebook we reproduce Figure 2 of the paper.

In [ ]:
import sys
import os
import json
import networkx as nx

# Since we are in a sub-package, ensure that the top-level package is on the Python path
sys.path.append(os.path.abspath(os.path.join('..', '..')))

# Now we can import modules from python_fbas
from python_fbas.fbas_graph import FBASGraph
from python_fbas.fbas_graph_analysis import is_fba_resilient_approx
from python_fbas.constellation.constellation import *
import python_fbas.constellation.config as config

# %load_ext line_profiler

In [ ]:

NUM_SAMPLES=50
for NUM_ORGS in [16, 25, 50, 75, 100, 150, 200]:
    # we'll cache results in 'data/tmp/':
    dir = f"data/tmp/{NUM_ORGS}"
    os.makedirs(dir, exist_ok=True)

    config.max_num_clusters = 4 if NUM_ORGS <= 50 else 3
    config.min_cluster_size = NUM_ORGS//(config.max_num_clusters+2) if NUM_ORGS <= 100 else NUM_ORGS//(config.max_num_clusters+1)

    # First, we create NUM_SAMPLES random single-universe FBA systems with 16 validators each.
    fbases = []
    for i in range(NUM_SAMPLES):
        f = f'{dir}/fbas_{i}.json'
        if os.path.exists(f):
            with open(f, 'r') as f:
                fbases.append(json.load(f))
            continue
        else:
            # random thresholds between 1/2 and 5/6 of the number of validators with 5 different thresholds
            fbas = random_single_universe_regular_fbas(NUM_ORGS, int(NUM_ORGS/2), int(5*NUM_ORGS/6), 5)
            fbases.append(fbas)
            with open(f, 'w') as f:
                json.dump(fbas, f)

    # For each FBAS, compute the constellation overlay:
    constellation_overlays = []
    for i, fbas in enumerate(fbases):
        f = f'{dir}/overlay_{i}.json'
        if os.path.exists(f):
            with open(f, 'r') as f:
                constellation_overlays.append(nx.node_link_graph(json.load(f)))
            continue
        else:
            overlay = constellation_overlay(fbas)
            constellation_overlays.append(overlay)
            # We save the overlay to a file for examination
            with open(f, 'w') as f:
                json.dump(nx.node_link_data(overlay), f)
    # print the average degree of the constellation overlays:
    constellation_avg_degrees = [sum(dict(overlay.degree()).values())/len(overlay) for overlay in constellation_overlays]
    print(constellation_avg_degrees)

    # We generate a random Erdos-Renyi graph with edge probability k/N for k from 1 to N until we find a graph that has diameter 2 and that is FBA-resilient.
    erdos_renyi = []
    for i, fbas in enumerate(fbases):
        fbas_graph:FBASGraph = single_universe_regular_fbas_to_fbas_graph(fbas)
        vertices = fbas_graph.vertices()
        validators = fbas_graph.validators
        overlay = {}
        f = f'{dir}/erdos_renyi_{i}.json'
        if os.path.exists(f):
            with open(f, 'r') as f:
                overlay = nx.node_link_graph(json.load(f))
            erdos_renyi.append(overlay)
        else:
            for d in range(10, len(validators)):
                overlay = nx.erdos_renyi_graph(len(validators), d/len(validators))
                # relabel with the validator names:
                overlay = nx.relabel_nodes(overlay, {i: v for i, v in enumerate(validators)})
                if is_fba_resilient_approx(fbas_graph, overlay) and nx.diameter(overlay) == 2:
                    break
            with open(f, 'w') as f:
                json.dump(nx.node_link_data(overlay), f)
            erdos_renyi.append(overlay)
    # print the average degree of the erdos-renyi graphs:
    erdos_avg_degrees = [sum(dict(overlay.degree()).values())/len(overlay) for overlay in erdos_renyi]
    print(erdos_avg_degrees)

[18.375, 18.375, 18.375, 18.375, 18.375, 19.0, 18.375, 18.375, 18.375, 18.375, 17.125, 18.375, 17.125, 18.375, 17.125, 17.125, 17.125, 18.375, 18.375, 18.375, 18.375, 18.375, 18.375, 18.375, 17.125, 18.375, 18.375, 18.375, 17.125, 18.375, 18.375, 18.375, 18.375, 18.375, 18.375, 18.375, 18.375, 18.375, 18.375, 19.0, 18.375, 17.0, 17.125, 17.125, 17.125, 17.125, 18.375, 18.375, 18.375, 18.375]
[22.916666666666668, 26.083333333333332, 23.25, 25.541666666666668, 28.875, 30.708333333333332, 24.625, 26.375, 23.083333333333332, 23.916666666666668, 22.666666666666668, 23.333333333333332, 21.583333333333332, 25.291666666666668, 24.458333333333332, 22.791666666666668, 23.666666666666668, 22.916666666666668, 28.125, 25.666666666666668, 23.916666666666668, 24.583333333333332, 25.625, 21.0, 21.708333333333332, 23.625, 22.208333333333332, 22.125, 22.958333333333332, 26.458333333333332, 24.0, 23.625, 23.208333333333332, 26.333333333333332, 25.5, 22.875, 22.375, 28.75, 22.166666666666668, 26.333333333

KeyboardInterrupt: 

In [ ]:
# TODO: now we have multiple series...
# print avg and variance of the degrees of the constellation overlays:
avg_degree = sum(constellation_avg_degrees)/len(constellation_avg_degrees)
variance = sum([(d - avg_degree)**2 for d in constellation_avg_degrees])/len(constellation_avg_degrees)
median = sorted(constellation_avg_degrees)[len(constellation_avg_degrees)//2]
print(avg_degree, variance, median)
# print avg, variance, and median of the degrees of the Erdos-Renyi graphs:
avg_degree = sum(erdos_avg_degrees)/len(erdos_avg_degrees)
variance = sum([(d - avg_degree)**2 for d in erdos_avg_degrees])/len(erdos_avg_degrees)
median = sorted(erdos_avg_degrees)[len(erdos_avg_degrees)//2]
print(avg_degree, variance, median)

138.23306666666664 426.87782570666707 153.0
117.50386666666664 67.23832727111109 115.86666666666666
